In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
# from google.colab import files 
import datetime


In [13]:
def my_ceil(predictions):
  for i in range(len(predictions)):
    if predictions[i]%1<=0.5:
      predictions[i] = int(predictions[i])
    else:
      predictions[i] = int(predictions[i]) + 1
  return predictions

#Export function 
def export( data_test, predictions):
  result_ = pd.DataFrame({'ID': data_test.ID, 'Score': my_ceil(predictions)}) 
  result_.to_csv('results_.csv', index = False) 
  
  

# Upload preprocessed dataset train and test  

In [4]:
test = pd.read_csv('https://raw.githubusercontent.com/nisrineha/Challenge_data_well_being_at_work/master/datasets/data_test_processed_holiday.csv')
train = pd.read_csv('https://raw.githubusercontent.com/nisrineha/Challenge_data_well_being_at_work/master/datasets/data_train_processed_holiday.csv')
test_with_ID = pd.read_csv('https://raw.githubusercontent.com/nisrineha/Challenge_data_well_being_at_work/master/datasets/test_input.csv')
test_with_date= pd.read_csv('https://raw.githubusercontent.com/nisrineha/Challenge_data_well_being_at_work/master/datasets/train_input.csv')

Creation of features and the target variable Score

In [5]:
y= train['Score']
train1= train
train1= train1.drop('Score', axis= 1)
X= train1


Spliting of the data 

In [18]:
X_train, X_test, y_train, y_test= train_test_split(X, y , test_size= 0.2, random_state= 42)


from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
clf = HistGradientBoostingClassifier(max_iter=10).fit(X_train, y_train)
x = test
y_pred = clf.predict(x)
# accuracy_score(y_test, y_pred)


# y_pred= pipes[-1].predict(x)
export(test_with_ID, y_pred)

##Pipeline method

Implement of pipeline method using different transformer: numeric and categorial 

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

#SimpleImputer fill any missing values 
#Scaler numeric transformer

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

#One hot encoder to transform categorial values into integers.

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

Transform the categorical features and numeric on train dataset and test 

In [8]:
#Select les columns numeric 
#Select les columns categoric 


integer_features = list(X.columns[X.dtypes == 'int64'])
continuous_features = list(X.columns[X.dtypes == 'float64'])
categorical_features = list(X.columns[X.dtypes == 'object'])
numeric_features = integer_features + continuous_features 


from sklearn.compose import ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

integer_features_test = list(test.columns[test.dtypes == 'int64'])
continuous_features_test = list(test.columns[test.dtypes == 'float64'])
categorical_features_test = list(test.columns[test.dtypes == 'object'])
numeric_features = integer_features + continuous_features 


from sklearn.compose import ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

##Model selection
In this section, we chose different classifier from sklearn, to get the best classifier for our dataset, we calculate the score thanks to the splitting of the dataset that we did before. 

In [11]:
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.experimental import enable_hist_gradient_boosting  # noqa
# now you can import normally from ensemble
from sklearn.ensemble import HistGradientBoostingClassifier
# from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="rbf", C=0.025, probability=True),
    DecisionTreeClassifier(),
    RandomForestClassifier(n_estimators= 400, max_depth=20),
    AdaBoostClassifier(),
    GradientBoostingClassifier(),
    HistGradientBoostingClassifier(max_iter=10) 
   
    ]
pipes= []
for classifier in classifiers:
    pipe = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', classifier)])
    pipe.fit(X_train, y_train)  
    pipes.append(pipe) 
    print(classifier)
    print("model score: %.3f" % pipe.score(X_test, y_test))


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')
model score: 0.519
SVC(C=0.025, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=True, random_state=None,
    shrinking=True, tol=0.001, verbose=False)
model score: 0.439
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')
model score: 0.591
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth

From the results above, we observe that the best classifiers are **GradientBoostingClassifier** and  **RandomForestClassifier**

we chose GradientBoostingClassifier for submission, we had our best score which is **0,6990**




In [14]:
x = test
y_pred= pipes[-1].predict(x)
export(test_with_ID, y_pred)

## Using pipeline in GridSearch 

In [15]:
param_grid = { 
    'classifier__n_estimators': [ 200, 300,  400, 500],
    'classifier__max_features': ['auto', 'sqrt', 'log2'],
    'classifier__max_depth' : [10, 20, 25, 30],
    'classifier__criterion' :['gini', 'entropy']}
from sklearn.model_selection import GridSearchCV
CV = GridSearchCV(HistGradientBoostingClassifier() , param_grid, n_jobs= 1)
                  
CV.fit(X_train, y_train)  
print(CV.best_params_)    
print(CV.best_score_)

C:\Users\Nisrine\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


ValueError: Invalid parameter classifier for estimator HistGradientBoostingClassifier(l2_regularization=0.0, learning_rate=0.1,
                               loss='auto', max_bins=256, max_depth=None,
                               max_iter=100, max_leaf_nodes=31,
                               min_samples_leaf=20, n_iter_no_change=None,
                               random_state=None, scoring=None, tol=1e-07,
                               validation_fraction=0.1, verbose=0). Check the list of available parameters with `estimator.get_params().keys()`.

we had from our grid search our best max depth and number of estimators for our randomforset model. 

In [0]:
#Fitting the classifier 
from sklearn.ensemble import RandomForestClassifier
rf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', RandomForestClassifier(n_estimators= 400, max_depth=20))])

In [0]:
rf.fit(X_train, y_train )

pipe = Pipeline(steps=[('preprocessor', preprocessor),
                  ('classifier',RandomForestClassifier(n_estimators= 400, max_depth=20) )])
pipe.fit(X_train, y_train)  
print(classifier)
print("model score: %.3f" % pipe.score(X_test, y_test))
x = test
y_pred= pipe.predict(x)
export(test_with_ID, y_pred)

QuadraticDiscriminantAnalysis(priors=None, reg_param=0.0,
                              store_covariance=False, tol=0.0001)
model score: 0.734


We submitted our results, but we had a score less than the one when we implemented **GradientBoostingClassifier**

In [19]:
 pip install scikit.ensemble 

SyntaxError: ignored

In [0]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
clf = HistGradientBoostingClassifier(max_iter=10).fit(x_train, y_train)
y_pred = clf.predict(x_test)
accuracy_score(y_test, y_pred)